In this notebook we will demonstrate how to use LangChain to perform [conversational retrieval](https://python.langchain.com/docs/use_cases/question_answering/chat_vector_db).

In a conversational question and answering scenario, users often pose follow-up questions related to the same topic, with the context being crucial to understand their queries. To address such cases effectively, we use the ConversationalRetrievalChain. Behind the scenes, this chain takes the user's question and converts it into a standalone query by considering the conversation history. Subsequently, it uses this standalone question to query the search service for relevant information.

In [ ]:
! pip install azure-identity

In this notebook we are utilizing an existing search index we already set up with Azure Cognitive Search. You can follow this [link](https://python.langchain.com/docs/integrations/vectorstores/azuresearch) to create your own search index. In our search index, the content vector field is named "contentVector", so we set it as an environment variable below.

In this notebook we are focusing on the ConversationalRetrievalChain only for demonstration purposes. If you use your own search index you might need to modify your queries so that it is relevant to the information in your index.

In [1]:
import os
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "contentVector"

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chains import ConversationalRetrievalChain


In [3]:
from dotenv import load_dotenv
load_dotenv("../llm.env")

True

In [4]:
from langchain.vectorstores.azuresearch import AzureSearch


model = os.getenv("OPENAI_DEPLOYMENT_EMBEDDING")
embeddings = OpenAIEmbeddings(deployment=model)
index_name = "testqa"
vector_store_address = os.getenv("COGSEARCH_ADDRESS")
vector_store_password = os.getenv("COGSEARCH_API_KEY")
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [5]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.output_key = "answer"

In [6]:
llm = AzureOpenAI(deployment_name=os.getenv("OPENAI_DEPLOYMENT_COMPLETION"), model_name=os.getenv("OPENAI_MODEL_COMPLETION"),temperature=0)
qa = ConversationalRetrievalChain.from_llm(llm, vector_store.as_retriever(search_kwargs={"k": 1}), memory=memory, return_generated_question=True)

In [7]:
query = "What did the astronaut Edgar Mitchell call Earth?"
result = qa({"question": query})

In [8]:
result["answer"]

' Edgar Mitchell called Earth a "sparkling blue and white jewel."'

In [9]:
query = "When and how did NASA make its first observation about it from the space?"
result = qa({"question": query})

In [10]:
result["answer"]

' NASA first observed Earth from space with the launch of Explorer 1 in 1960.'

As you can see, if we had query the search service using the user's question directly, it is not clear wht the user is reffering to by "its" and "it". As you can see below, the generated standalone question makes it more cleary, and enables for more efficient and robust queries.

In [11]:
result["generated_question"]

' When and how did NASA first observe Earth from space?'

When the user asks a question on a different topic, the generated question still reflects the user's question correctly.

In [12]:
query = "Why can't we see volcanic plumes with our eyes?"
result = qa({"question": query})

In [13]:
print(result["answer"])
print(result["generated_question"])

 Volcanic plumes are not visible to the naked eye because they are typically invisible in the electromagnetic spectrum. However, satellites can use infrared to distinguish the plumes from ice and clouds.
 Why are volcanic plumes not visible to the naked eye?
